In [21]:
import numpy as np
import open3d as o3d
import os
from makegripper_points import  plot_gripper_pro_max
from scipy.spatial.transform import Rotation as R

# Include your original plot_gripper_pro_max function here or import it if it's from another module

def visualize_selected_grasps(data_dir, grasp_indices):
    # Load saved data
    cloud_path = os.path.join(data_dir, "cloud.ply")
    grasps_path = os.path.join(data_dir, "grasps.npy")
    
    if not os.path.exists(cloud_path) or not os.path.exists(grasps_path):
        print("Error: cloud.ply or grasps.npy not found in the directory.")
        return

    cloud = o3d.io.read_point_cloud(cloud_path)
    grasp_data = np.load(grasps_path, allow_pickle=True).item()

    translations = grasp_data['translations']
    rotations = grasp_data['rotations']
    widths = grasp_data['widths']
    heights = grasp_data['heights']
    scores = grasp_data['scores']
    gripper_points = grasp_data['gripper_points']

    pcds = []
    dots = []

    for idx in grasp_indices:
        if idx >= len(translations):
            print(f"Index {idx} out of range.")
            continue

        t = translations[idx]
        R_mat = rotations[idx]
        width = widths[idx]
        height = heights[idx]
        score = scores[idx]
        gripper_point = gripper_points[idx]

        # Gripper mesh
        gripper_pcd, _ = plot_gripper_pro_max(t, R_mat, width=width, depth=height, score=score, color=[0, 1, 0])
        pcds.append(gripper_pcd)

        # Red dots
        for p in gripper_point:
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
            sphere.translate(p)
            sphere.paint_uniform_color([1, 0, 0])
            dots.append(sphere)


###
        R_mat_XG = R_mat.copy() #xarm gripper
        R_mat_XG[:, [0, 2]] = R_mat_XG[:, [2, 0]]  # Swap X and Z axes in real -life gripper
        R_mat_XG[:, 1] *= -1  

        #Move gripper backwards along its local z-axis by 0.15 meters
        z_axis = R_mat_XG[:, 2]  # Local z-axis (gripper heading)
        t_moved = t - 0.15 * z_axis  # Move backwards by 0.15

        xarm_gripT = np.eye(4)  
        xarm_gripT[:3, :3] = R_mat_XG
        xarm_gripT[:3, 3] = t_moved
###

        rpy = R.from_matrix(R_mat_XG).as_euler('xyz', degrees=True) 

        gaxis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
        gaxis.rotate(R_mat, center=(0, 0, 0))
        gaxis.translate(t)
        pcds.append(gaxis)

        print(f"\nGrasp Index: {idx}")
        print(f"  Score: {score:.4f}")
        print(f"  Translation (x, y, z): {t_moved*1000}")
        print(f"  Rotation (roll, pitch, yaw) [deg]: {rpy}")
        print(f"  Width: {width:.4f}  | Depth: {height:.4f}")


    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
    o3d.visualization.draw_geometries([cloud, axis] + pcds + dots)




visualize_selected_grasps('./pavanoutput6', grasp_indices=[0])



Grasp Index: 0
  Score: 0.3628
  Translation (x, y, z): [541.60509823 107.42443968 272.46122458]
  Rotation (roll, pitch, yaw) [deg]: [-174.01676166   -1.33694194 -107.74377875]
  Width: 0.0685  | Depth: 0.0300
